In [44]:
import os
import numpy as np
import pandas as pd
from sklearn.svm import SVR
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.multioutput import MultiOutputRegressor
from sklearn.metrics import make_scorer
from sklearn.model_selection import train_test_split

In [45]:
import tensorflow as tf
from tensorflow.keras import Input, Model, layers
import matplotlib.pyplot as plt
from PIL import Image

In [46]:
# To find mean of an integer list
def Average(lst):
    return sum(lst) / len(lst)

In [47]:
#This is used to remove spaces from a string
def remove(string):
    return string.replace(" ", "")

In [48]:
# Euclidean distance is defined
# def euc(a, b):
#     dist = np.sqrt(np.sum(np.square(a - b), axis=1))
#     mean = dist.mean()
#     return mean

def euc(a, b):
    return np.sqrt(np.sum(np.square(a - b), axis=1))

In [159]:
mit_model = tf.keras.models.load_model('../../checkpoints/new_mod/gs/rlrop100/epoch-99-vl-2.371.ckpt')
in_layer = 're_lu_2'
out_layer = 'dense_7'
pen_model = Model(inputs=mit_model.get_layer(in_layer).input, 
                  outputs=mit_model.get_layer(out_layer).output)

pen_model.summary()

Model: "model_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_3 (InputLayer)        [(None, 4)]               0         
                                                                 
 re_lu_2 (ReLU)              (None, 4)                 0         
                                                                 
 dense_7 (Dense)             (None, 2)                 10        
                                                                 
Total params: 10
Trainable params: 10
Non-trainable params: 0
_________________________________________________________________


In [160]:
# IDs = ['03183','01877','01326','03140','02091','02301','02240','00382','02833','02078']
IDs = ['00503','01866','02459','01816','03004','03253','01231','02152','02015','01046']
# ID = '03183'

In [161]:
whole_results = []

In [171]:
%%capture testmed

results = []

for ID in IDs:
    # csv_path = '../../CSVs/MIT_unique/unique30_' + ID + '.csv'
    csv_path = '../../CSVs/google_all/' + ID + '.csv'
    # csv_path = '../../CSVs/MIT_all/'+ ID + '.csv'

    Model_1=pd.read_csv(csv_path)

    train, test = train_test_split(Model_1, test_size=0.334,random_state=42, shuffle=False)

    a_test=test['Penultimate_Output']
    b_test=test['GT_Value']


    for i in range(0,len(a_test)):
        lis=[]
        a_test.iloc[i]=remove(a_test.iloc[i])
        x=a_test.iloc[i]
        a=x.split(',')
        x=a[0]
        x=x[9:]
        x1=a[1]

        x2=a[2]

        x3=a[3]
        x3=x3[0:-2]

        lis.append(float(x))
        lis.append(float(x1))
        lis.append(float(x2))
        lis.append(float(x3))
        a_test.iloc[i]=lis

    for i in range(0,len(b_test)):
        lis=[]
        x=b_test.iloc[i]
        a=x.split(',')
        x1=a[0]
        x1=x1[9:]
        x2=a[1]
        x2=x2[0:-2]
        lis.append(float(x1))
        lis.append(float(x2))
        b_test.iloc[i]=lis

    for i in range(0,len(a_test)):
        a_test.iloc[i]=np.array(a_test.iloc[i])
    for i in range(0,len(b_test)):
        b_test.iloc[i]=np.array(b_test.iloc[i])

    a_test = a_test.apply(pd.Series)
    b_test = b_test.apply(pd.Series)
    x_test = a_test.to_numpy()
    y_test = b_test.to_numpy()
    
    test_dataset = tf.data.Dataset.from_tensor_slices((x_test, y_test))
    test_dataset = test_dataset.batch(1)
    
    preds, gt = [], []

    for x_test,y_test in test_dataset:
        pred = pen_model.predict(x_test)
        preds.extend(pred)
        gt.extend(y_test)
        
    preds = np.array(preds)
    pts = np.unique(gt, axis=0)
    gt = np.array(gt)
    dist = euc(preds, gt)
    MED = round(dist.mean(), 3)
    # print("Mean Euclidean Distance: ", MED) 
    
    results.append(MED)

In [172]:
whole_results.append(results)

In [173]:
whole_results

[[1.528, 1.196, 1.313, 1.096, 1.358, 1.243, 1.331, 1.573, 1.284, 1.227],
 [1.207, 1.478, 1.658, 0.987, 1.063, 1.305, 1.401, 1.386, 1.295, 1.289],
 [1.539, 1.213, 1.293, 1.093, 1.368, 1.257, 1.323, 1.587, 1.275, 1.233],
 [1.232, 1.467, 1.657, 1.002, 1.09, 1.253, 1.494, 1.39, 1.258, 1.344]]

In [174]:
whole_results = np.array(whole_results)
whole_results_np = whole_results.transpose()
whole_results_np

array([[1.528, 1.207, 1.539, 1.232],
       [1.196, 1.478, 1.213, 1.467],
       [1.313, 1.658, 1.293, 1.657],
       [1.096, 0.987, 1.093, 1.002],
       [1.358, 1.063, 1.368, 1.09 ],
       [1.243, 1.305, 1.257, 1.253],
       [1.331, 1.401, 1.323, 1.494],
       [1.573, 1.386, 1.587, 1.39 ],
       [1.284, 1.295, 1.275, 1.258],
       [1.227, 1.289, 1.233, 1.344]])

In [175]:
df = pd.DataFrame(whole_results_np, columns=['73T', '73F', '23T','23F'])

In [176]:
df.loc[len(df.index)] = [round(Average(whole_results[0]),2) , round(Average(whole_results[1]),2) , round(Average(whole_results[2]),2), round(Average(whole_results[3]),)] 

In [177]:
mean_4versions = list(df.mean(axis=1))
for i in range(len(mean_4versions)):
    mean_4versions[i] = round(mean_4versions[i], 2)

mean_4versions

[1.38, 1.34, 1.48, 1.04, 1.22, 1.26, 1.39, 1.48, 1.28, 1.27, 1.24]

In [178]:
df.insert(4, 'Mean of all 4', mean_4versions)
df

,73T,73F,23T,23F,Mean of all 4
0,1.528,1.207,1.539,1.232,1.38
1,1.196,1.478,1.213,1.467,1.34
2,1.313,1.658,1.293,1.657,1.48
3,1.096,0.987,1.093,1.002,1.04
4,1.358,1.063,1.368,1.090,1.22
5,1.243,1.305,1.257,1.253,1.26
6,1.331,1.401,1.323,1.494,1.39
7,1.573,1.386,1.587,1.390,1.48
8,1.284,1.295,1.275,1.258,1.28
9,1.227,1.289,1.233,1.344,1.27


In [179]:
df.to_csv('goolgle-all-MED.csv', index = False)